In [ ]:
import sys
import pandas as pd
from pandas import read_sql_query, read_sql_table
import tables
import sqlite3
import sqlalchemy
import matplotlib.pyplot as plt

In [ ]:
####################################################################################
#   description     : read sqlite file
#   Args:
#       dbfile      : database file 
####################################################################################
def read_sqlite(dbfile):
    with sqlite3.connect(dbfile) as dbcon:
        tables = list(read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", dbcon)['name'])
        out = {tbl : read_sql_query(f"SELECT * from {tbl}", dbcon) for tbl in tables}
    return out


####################################################################################
#   description     : For all variants (network/image size in this case), 
#                     load all stat files from single test
#   Args:
#       master      : master dictionary resembling the folder structure: 
#                     [root]->[ test_case_# ]->[ $(variant)_$(stat).csv ] 
#       test_no     : specify a single test case
#       variants    : img_size?
#       stat_file   : stat file name (string)
####################################################################################
def singleTest_allVariants_stats(master, test, variant_prefix, variants, stat_file):
    db = {}
    for v in variants:
        db[v] = master[test][str(variant_prefix+v+"_"+stat_file+'.csv')]
    return db



####################################################################################
#   description     : For all variants (network/image size in this case), 
#                   load all stat files from all the test cases          
#   Args:
#       master       : master dictionary resembling the folder structure: 
#                      [root]->[ test_case_# ]->[ $(variant)_$(stat).csv ] 
#       test_no      : specify a single test case
#       variants     : img_size?
#       stat_file   : stat file name (string)
####################################################################################
def allTests_allVariants_stats(master, tests, variant_prefix, variants, stat_file):
    file_ext  = '.csv'
    db = {}
    temp = {}

    for test_no in tests:
        for v in variants:
            temp[v] = master[test_no][str(variant_prefix+v+"_"+stat_file+file_ext)]
        db[test_no] = temp
    return db



####################################################################################
#   description     : add suffix to all the columns of a dataframe         
#   Args:
#       df          : dataframe  
#       sfx         : suffix
####################################################################################
# 
def add_suffix(df,sfx, ignore):
    df.columns = df.columns.map(lambda x: str(x) if x == ignore else (str(x) + sfx))



####################################################################################
#   description     : load all metrics from all the tests, average the values         
#   Args:
#       src         : Source dataframe containing all tests and variants
#                      df [ variant_# ] [ test_# ] 
#       tests       : specify a list of test folder names of type string.
#       variants    : img_size?
#       grpByVal    : the key/index. 
####################################################################################
def get_avg_of_all_tests(src, tests= master.keys(), variants=network_sizes, grpByVal='Name'):
    avg = {}
    for v in variants:
        avg[v] = pd.DataFrame()
        for test in tests:
            avg[v] = pd.concat([avg[v],src[test][v]])
        avg[v] = avg[v].groupby(grpByVal,as_index=False).mean()
    return avg


####################################################################################
#   description     : Extract a single metric from all the variants for plotting
#   Args:
#       src         : Source dataframe containing all tests and variants
#                      df [ variant_# ] 
#       x           : key/index value that forms the X-axis in the plot.
#       y           : metric to view/compare against other variants
#       variants    : img_size?
####################################################################################
def get_single_metric_all_variants(src=None, x='Time',  y=None, variants=network_sizes):
    data = src[variants[0]][[x]]
    for v in variants:
        add_suffix(src[v], str('_'+v), ignore=x)
        data = pd.concat([data, src[v][[str(y+'_'+v)]]], axis=1)
    return data


<H1> Combined stats from all the frame-inferences


<H3> Test VST2: small video

In [ ]:
#Get all files
import os

#define base paths
nsys_rep_base = '../nsys_reports/vst2'
test_folders = ['test1','test2','test3']
network_sizes = ['64','128','256','512','1024'] #network input sizes

#master dataframe = dictionary of all stat files
master = {}

for folder in test_folders:
    temp_path = os.path.join(nsys_rep_base,folder,'stats')
    temp={}
    for f in os.listdir(path=os.path.join(nsys_rep_base,folder,'stats')):
        temp[str(f)] = pd.read_csv(os.path.join(nsys_rep_base,folder,'stats',f))
    master[str(folder)] = temp

stats_types = master['test1'].keys()


In [98]:
def show_n_save_graphs(test_no, stat_file, x, y):
    nvtx_sum = allTests_allVariants_stats(master=master,   tests= master.keys(),    variant_prefix='size_', variants=network_sizes, stat_file=stat_file)
    nvtx_avg = get_avg_of_all_tests(src=nvtx_sum, tests=master.keys(), variants=network_sizes, grpByVal=x)
    nvtx_total_time = get_single_metric_all_variants(nvtx_avg, x=x, y=y)

    ax = nvtx_total_time.plot(kind='bar',x=_x, title=_y, ylabel='time (ns)')
    plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
    ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

    fig = ax.get_figure()
    fig.savefig('figures/vst{test_no}_{stat_file}_{y}.jpg',bbox_inches='tight')
    fig = ax.get_figure()

<H3> NVTX: total times (ns)

In [ ]:
_x = 'Name'
_y = 'Total Time (ns)'
_stat_file = 

show_n_save_graphs('cuda_api_sum', 'Name', )
nvtx_sum = allTests_allVariants_stats(master=master,   tests= master.keys(),    variant_prefix='size_', variants=network_sizes, stat_file=)
nvtx_avg = get_avg_of_all_tests(src=nvtx_sum, tests=master.keys(), variants=network_sizes, grpByVal=_x)
nvtx_total_time = get_single_metric_all_variants(nvtx_avg, x=_x, y=_y)

ax = nvtx_total_time.plot(kind='bar',x=_x, title=_y, ylabel='time (ns)')
plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

fig = ax.get_figure()
fig.savefig('figures/nvtx_vst2_total_time.jpg',bbox_inches='tight')
fig = ax.get_figure()


<H3> NVTX: Median times (ns)

In [ ]:
_x = 'Name'
_y = 'Med (ns)'

nvtx_sum = allTests_allVariants_stats(master=master,   tests= master.keys(),    variant_prefix='size_', variants=network_sizes, stat_file='cuda_api_sum')
nvtx_avg = get_avg_of_all_tests(src=nvtx_sum, tests=master.keys(), variants=network_sizes, grpByVal=_x)
nvtx_total_time = get_single_metric_all_variants(nvtx_avg, x=_x, y=_y)

ax = nvtx_total_time.plot(kind='bar',x=_x, title=_y, ylabel='time (ns)',log=True)
plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

fig = ax.get_figure()
fig.savefig('figures/nvtx_vst2_median_time.jpg',bbox_inches='tight')
fig = ax.get_figure()


<H3> NVTX: number of calls

In [ ]:
_x = 'Name'
_y = 'Num Calls'

nvtx_sum = allTests_allVariants_stats(master=master,   tests= master.keys(),    variant_prefix='size_', variants=network_sizes, stat_file='cuda_api_sum')
nvtx_avg = get_avg_of_all_tests(src=nvtx_sum, tests=master.keys(), variants=network_sizes, grpByVal=_x)
nvtx_total_time = get_single_metric_all_variants(nvtx_avg, x=_x, y=_y)

ax = nvtx_total_time.plot(kind='bar',x=_x, title=_y, ylabel=_y, log=True)
plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

fig = ax.get_figure()
fig.savefig(f'figures/nvtx_vst2_{_y}.jpg',bbox_inches='tight')
fig = ax.get_figure()


<H3> NVTX: Avg time

In [ ]:
_x = 'Name'
_y = 'Avg (ns)'

nvtx_sum = allTests_allVariants_stats(master=master,   tests= master.keys(),    variant_prefix='size_', variants=network_sizes, stat_file='cuda_api_sum')
nvtx_avg = get_avg_of_all_tests(src=nvtx_sum, tests=master.keys(), variants=network_sizes, grpByVal=_x)
nvtx_total_time = get_single_metric_all_variants(nvtx_avg, x=_x, y=_y)

ax = nvtx_total_time.plot(kind='bar',x=_x, title=_y, ylabel=_y, log=True)
plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

fig = ax.get_figure()
fig.savefig(f'figures/nvtx_vst2_{_y}.jpg',bbox_inches='tight')
fig = ax.get_figure()


In [ ]:
_stat_file = 'cuda_gpu_mem_size_sum'
_x = 'Operation'
_y = 'Total (MB)'

nvtx_sum = allTests_allVariants_stats(master=master,   tests= master.keys(),    variant_prefix='size_', variants=network_sizes, stat_file=_stat_file)
nvtx_avg = get_avg_of_all_tests(src=nvtx_sum, tests=master.keys(), variants=network_sizes, grpByVal=_x)
nvtx_total_time = get_single_metric_all_variants(nvtx_avg, x=_x, y=_y)

ax = nvtx_total_time.plot(kind='bar',x=_x, title=_y, ylabel=_y)
plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

fig = ax.get_figure()
fig.savefig(f'figures/vst2_{_stat_file}_{_y}.jpg',bbox_inches='tight')
fig = ax.get_figure()


In [ ]:
_stat_file = 'cuda_gpu_mem_size_sum'
_x = 'Operation'
_y = 'Avg (MB)'

nvtx_sum = allTests_allVariants_stats(master=master,   tests= master.keys(),    variant_prefix='size_', variants=network_sizes, stat_file=_stat_file)
nvtx_avg = get_avg_of_all_tests(src=nvtx_sum, tests=master.keys(), variants=network_sizes, grpByVal=_x)
nvtx_total_time = get_single_metric_all_variants(nvtx_avg, x=_x, y=_y)

ax = nvtx_total_time.plot(kind='bar',x=_x, title=_y, ylabel=_y)
plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

fig = ax.get_figure()
fig.savefig(f'figures/vst2_{_stat_file}_{_y}.jpg',bbox_inches='tight')
fig = ax.get_figure()


In [ ]:
_stat_file = 'osrt_sum'
_x = 'Name'
_y = 'Time (%)'

nvtx_sum = allTests_allVariants_stats(master=master,   tests= master.keys(),    variant_prefix='size_', variants=network_sizes, stat_file=_stat_file)
nvtx_avg = get_avg_of_all_tests(src=nvtx_sum, tests=master.keys(), variants=network_sizes, grpByVal=_x)
nvtx_total_time = get_single_metric_all_variants(nvtx_avg, x=_x, y=_y)

ax = nvtx_total_time.plot(kind='bar',x=_x, title=_y, ylabel=_y, log=True)
plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

fig = ax.get_figure()
fig.savefig(f'figures/vst2_{_stat_file}_{_y}.jpg',bbox_inches='tight')
fig = ax.get_figure()


In [ ]:
_stat_file = 'osrt_sum'
_x = 'Name'
_y = 'Num Calls'

nvtx_sum = allTests_allVariants_stats(master=master,   tests= master.keys(),    variant_prefix='size_', variants=network_sizes, stat_file=_stat_file)
nvtx_avg = get_avg_of_all_tests(src=nvtx_sum, tests=master.keys(), variants=network_sizes, grpByVal=_x)
nvtx_total_time = get_single_metric_all_variants(nvtx_avg, x=_x, y=_y)

ax = nvtx_total_time.plot(kind='bar',x=_x, title=_y, ylabel=_y, log=True)
plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

fig = ax.get_figure()
fig.savefig(f'figures/vst2_{_stat_file}_{_y}.jpg',bbox_inches='tight')
fig = ax.get_figure()


In [ ]:
_stat_file = 'osrt_sum'
_x = 'Name'
_y = 'Avg (ns)'

nvtx_sum = allTests_allVariants_stats(master=master,   tests= master.keys(),    variant_prefix='size_', variants=network_sizes, stat_file=_stat_file)
nvtx_avg = get_avg_of_all_tests(src=nvtx_sum, tests=master.keys(), variants=network_sizes, grpByVal=_x)
nvtx_total_time = get_single_metric_all_variants(nvtx_avg, x=_x, y=_y)

ax = nvtx_total_time.plot(kind='bar',x=_x, title=_y, ylabel=_y)
plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

fig = ax.get_figure()
fig.savefig(f'figures/vst2_{_stat_file}_{_y}.jpg',bbox_inches='tight')
fig = ax.get_figure()


In [ ]:
_stat_file = 'cuda_gpu_mem_size_sum'
_x = 'Operation'
_y = 'Total (MB)'

nvtx_sum = allTests_allVariants_stats(master=master,   tests= master.keys(),    variant_prefix='size_', variants=network_sizes, stat_file=_stat_file)
nvtx_avg = get_avg_of_all_tests(src=nvtx_sum, tests=master.keys(), variants=network_sizes, grpByVal=_x)
nvtx_total_time = get_single_metric_all_variants(nvtx_avg, x=_x, y=_y)

ax = nvtx_total_time.plot(kind='bar',x=_x, title=_y, ylabel=_y)
plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

fig = ax.get_figure()
fig.savefig(f'figures/vst2_{_stat_file}_{_y}.jpg',bbox_inches='tight')
fig = ax.get_figure()


In [ ]:
_stat_file = 'cuda_gpu_mem_size_sum'
_x = 'Operation'
_y = 'Count'

nvtx_sum = allTests_allVariants_stats(master=master,   tests= master.keys(),    variant_prefix='size_', variants=network_sizes, stat_file=_stat_file)
nvtx_avg = get_avg_of_all_tests(src=nvtx_sum, tests=master.keys(), variants=network_sizes, grpByVal=_x)
nvtx_total_time = get_single_metric_all_variants(nvtx_avg, x=_x, y=_y)

ax = nvtx_total_time.plot(kind='bar',x=_x, title=_y, ylabel=_y)
plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

fig = ax.get_figure()
fig.savefig(f'figures/vst2_{_stat_file}_{_y}.jpg',bbox_inches='tight')
fig = ax.get_figure()


In [ ]:
_stat_file = 'cuda_gpu_mem_time_sum'
_x = 'Operation'
_y = 'Count'

nvtx_sum = allTests_allVariants_stats(master=master,   tests= master.keys(),    variant_prefix='size_', variants=network_sizes, stat_file=_stat_file)
nvtx_avg = get_avg_of_all_tests(src=nvtx_sum, tests=master.keys(), variants=network_sizes, grpByVal=_x)
nvtx_total_time = get_single_metric_all_variants(nvtx_avg, x=_x, y=_y)

ax = nvtx_total_time.plot(kind='bar',x=_x, title=_y, ylabel=_y)
plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

fig = ax.get_figure()
fig.savefig(f'figures/vst2_{_stat_file}_{_y}.jpg',bbox_inches='tight')
fig = ax.get_figure()


In [ ]:
_stat_file = 'cuda_gpu_mem_time_sum'
_x = 'Operation'
_y = 'Total Time (ns)'

nvtx_sum = allTests_allVariants_stats(master=master,   tests= master.keys(),    variant_prefix='size_', variants=network_sizes, stat_file=_stat_file)
nvtx_avg = get_avg_of_all_tests(src=nvtx_sum, tests=master.keys(), variants=network_sizes, grpByVal=_x)
nvtx_total_time = get_single_metric_all_variants(nvtx_avg, x=_x, y=_y)

ax = nvtx_total_time.plot(kind='bar',x=_x, title=_y, ylabel=_y)
plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

fig = ax.get_figure()
fig.savefig(f'figures/vst2_{_stat_file}_{_y}.jpg',bbox_inches='tight')
fig = ax.get_figure()


In [ ]:
master['test1']['size_128_cuda_gpu_mem_time_sum.csv']

In [ ]:
stats_types

In [ ]:
#nvtx_avg = {}
#for sz in network_sizes:
#    nvtx_avg[sz] = pd.concat([nvtx_sum['test1'][sz], nvtx_sum['test2'][sz], nvtx_sum['test3'][sz]]).groupby('Range', as_index=False).mean()

In [ ]:
#nvtx_list = ['nvtx_64','nvtx_128','nvtx_256','nvtx_512','nvtx_1024']
#nvtx = {}
#    
#for sz in network_sizes:
#    nvtx[sz] = nvtx_avg[sz][['Time (%)','Total Time (ns)','Instances','Avg (ns)','Med (ns)','Range']]
#    nvtx[sz] = nvtx[sz].set_index('Range')
#    add_suffix(nvtx[sz],str('_sz'+sz), ignore='Range')
#    nvtx[sz].reset_index(inplace=True)

In [ ]:
##########################################
#col = 'Total Time (ns)'
#
#nvtx_tot_time = pd.concat([
#    nvtx['64'][['Range',    col+'_sz64']],
#    nvtx['128'][            col+'_sz128'],
#    nvtx['256'][            col+'_sz256'],
#    nvtx['512'][            col+'_sz512'],
#    nvtx['1024'][           col+'_sz1024']
#    ],axis=1)
#nvtx_tot_time

In [ ]:
ax = nvtx_total_time.plot(kind='bar',x='Range', title='total time')
plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
ax.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)

#ax1 = cas_num_calls.plot(kind='bar',x='Name', title='number of calls',log=True)
#plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
#ax1.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)
#
#ax2 = cas_time_perct.plot(kind='bar',x='Name', title='time (%)',log=True)
#plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right');
#ax2.legend(loc='upper center', bbox_to_anchor=(1.22, 1.02), ncol=1, fancybox=False, shadow=True)
#
fig = ax.get_figure()
fig.savefig('figures/nvtx_vst2_tot_time.jpg',bbox_inches='tight')
fig = ax.get_figure()
#fig.savefig('figures/cas_num_of_calls.jpg',bbox_inches='tight')
#fig = ax.get_figure()
#fig.savefig('figures/cas_percent.jpg',bbox_inches='tight')

<H2> Test3 </H2>